In [21]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
def generate(text_in, tok_in, mod_in):
    tok_text = tok_in(text_in, return_tensors='pt').to('cuda:0')
    gen_text = mod_in.generate(**tok_text, max_new_tokens=512)
    dec_text = tok_in.decode(gen_text[0], skip_special_tokens=True)
    return dec_text

In [3]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.3")


baseline_model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.3", torch_dtype=torch.float32)
# rlhf_model = AutoModelForCausalLM.from_pretrained("../Reward Model/model/", torch_dtype=torch.float32)

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v0.3 and are newly initialized: ['model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rota

In [4]:
baseline_model.to('cuda:0')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32003, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()


In [5]:
df = pd.read_feather('../mini_codenet/data/split/evaluate_train.ftr')
df.head()

,level_0,index,submission_id,problem_id,language,filename_ext,status,cpu_time,memory,code_size,accuracy,solution,problem_statement
0,1619278,5172,s373894550,p03202,C++,cpp,Wrong Answer,2103.0,1916.0,2741,None,#include <bits/stdc++.h>\nusing namespace std;...,Score : 700 points \n Problem Statement There ...
1,771169,5835,s440823744,p03008,C++,cpp,Wrong Answer,176.0,256.0,1378,None,#include <bits/stdc++.h>\n\nusing namespace st...,Score : 600 points \n Problem Statement The sq...
2,1654494,7516,s716084373,p03943,Python,py,Wrong Answer,17.0,2940.0,88,None,"a, b, c = map(int, input().split())\n\nif a ==...",Score : 100 points \n Problem Statement Two st...
3,758900,5870,s029981993,p03047,Python,py,Accepted,17.0,2940.0,43,None,"n,k = map(int,input().split())\nprint(n-k+1)",Score : 100 points \n Problem Statement Snuke ...
4,2792852,1786,s487610428,p03852,Python,py,Accepted,168.0,38384.0,114,None,"c = input()\n\nif c == ""a"" or c == ""i"" or c ==...",Score : 100 points \n Problem Statement Given ...


In [6]:
eval_df = pd.read_feather('./file.feather')
eval_df.head()

,a,b
0,p02402,5\n10 1 5 4 17
1,p03316,12
2,p02995,4 9 2 3
3,p03129,3 2
4,p02612,1900


In [16]:
print(len(eval_df))

50


In [29]:
num_compiled = 0
num_checked = 0
dist = len(os.listdir('./baseline_files/'))
print(dist)
for pid in tqdm(set(eval_df['a'].values)):
    if num_checked >= dist-1:
        instance = df.loc[(df['status'] == 'Accepted') & (df['language'] == 'Python') & (df['problem_id'] == pid)]
        
        prompt = instance.iloc[0]['problem_statement']
        prompt = prompt.replace('\n', '')
        lang = instance.iloc[0]['language']
        # ------ baseline -------
        formatted_prompt = (f"<|im_start|>user\nGenerate the correct {lang} code to answer the following prompt that can also takens input and prints out an answer:{prompt}<|im_end|>\n<|im_start|>assistant\n")
        generated = generate(formatted_prompt, tokenizer, baseline_model)
        try:
            baseline_generated = re.search(f'```{lang.lower()}(\n|.)*```', generated).group()
        except:
            baseline_generated = re.search(f'```{lang.lower()}(\n|.)*```', generated+'```').group()
        #print(baseline_generated)
        try:
            open(f'./baseline_files/{num_checked}.txt', 'w', encoding="utf-8").write(baseline_generated)
        except:
            open(f'./baseline_files/{num_checked}.txt', 'w', encoding="utf-8").write(generated)
        try:
            x = compile(baseline_generated[3+len(lang):-3], 'test', 'exec')
            num_compiled += 1
            num_checked += 1
        except:
            num_checked += 1
            continue
    else:
        num_checked += 1
    #out = exec(x)
    #print(out)
# ------ finetuned -------
# formatted_prompt = (f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistantPython\n") # update this with newest model
# generated = generate(formatted_prompt, tokenizer, tuned_model)
# fine

28


100%|██████████| 50/50 [13:48<00:00, 16.58s/it]


In [30]:
tuned_model = AutoModelForCausalLM.from_pretrained("../finetuning/hf_model/", torch_dtype=torch.float32)
tuned_model.to('cuda:0')

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at ../finetuning/hf_model/ and are newly initialized: ['model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32003, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()


In [34]:
num_compiled = 0
num_checked = 0
dist = len(os.listdir('./finetune_files/'))
print(dist)
for pid in tqdm(set(eval_df['a'].values)):
    if num_checked >= dist-1:
        instance = df.loc[(df['status'] == 'Accepted') & (df['language'] == 'Python') & (df['problem_id'] == pid)]
        
        prompt = instance.iloc[0]['problem_statement']
        prompt = prompt.replace('\n', '')
        lang = instance.iloc[0]['language']
        # ------ finetune -------
        formatted_prompt = (f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n```python\n")        
        generated = generate(formatted_prompt, tokenizer, tuned_model)
        print(generated)
        try:
            finetune_generated = re.search(f'```{lang.lower()}(\n|.)*```', generated).group()
        except:
            finetune_generated = re.search(f'```{lang.lower()}(\n|.)*```', generated+'```').group()
        #print(baseline_generated)
        try:
            open(f'./finetune_files/{num_checked}.txt', 'w', encoding="utf-8").write(finetune_generated)
        except:
            open(f'./finetune_files/{num_checked}.txt', 'w', encoding="utf-8").write(generated)
        try:
            x = compile(finetune_generated[3+len(lang):-3], 'test', 'exec')
            num_compiled += 1
            num_checked += 1
        except:
            num_checked += 1
            continue
    else:
        num_checked += 1

0


  2%|▏         | 1/50 [00:37<30:36, 37.48s/it]

user
Score : 300 points  Problem Statement Takahashi, who lives on the number line, is now at coordinate X . He will make exactly K moves of distance D in the positive or negative direction.  More specifically, in one move, he can go from coordinate x to x + D or x - D .  He wants to make K moves so that the absolute value of the coordinate of the destination will be the smallest possible.  Find the minimum possible absolute value of the coordinate of the destination.  Constraints  -10^{15} \leq X \leq 10^{15}  1 \leq K \leq 10^{15}  1 \leq D \leq 10^{15}  All values in input are integers.  Input Input is given from Standard Input in the following format: ``` X K D ``` Output Print the minimum possible absolute value of the coordinate of the destination.  Sample Input 1 ```6 2 4``` Sample Output 1 ```2``` Takahashi is now at coordinate 6 . It is optimal to make the following moves:  Move from coordinate 6 to ( 6 - 4 = ) 2 .  Move from coordinate 2 to ( 2 - 4 = ) -2 .  Here, the absolut

  4%|▍         | 2/50 [01:14<29:56, 37.42s/it]

user
Score : 100 points  Problem Statement Takahashi loves takoyaki - a ball-shaped snack.  With a takoyaki machine, he can make at most X pieces of takoyaki at a time, taking T minutes regardless of the number of pieces to make.  How long does it take to make N takoyaki?  Constraints  1 \leq N,X,T \leq 1000  All values in input are integers.  Input Input is given from Standard Input in the following format: ``` N X T ``` Output Print an integer representing the minimum number of minutes needed to make N pieces of takoyaki.  Sample Input 1 ```20 12 6``` Sample Output 1 ```12``` He can make 12 pieces of takoyaki in the first 6 minutes and 8 more in the next 6 minutes, so he can make 20 in a total of 12 minutes.  Note that being able to make 12 in 6 minutes does not mean he can make 2 in 1 minute.  Sample Input 2 ```1000 1 1000``` Sample Output 2 ```1000000``` It seems to take a long time to make this kind of takoyaki. 
 assistant
```python
import sys

N, X, T = map(int, input().split())

...

user
Score : 100 points  Problem Statement Determine if we can choose K different integers between 1 and N (inclusive) so that no two of them differ by 1 .  Constraints  1\leq N,K\leq 100  N and K are integers.  Input Input is given from Standard Input in the following format: ``` N K ``` Output If we can choose K integers as above, print YES ; otherwise, print NO .  Sample Input 1 ```3 2``` Sample Output 1 ```YES``` We can choose 1 and 3 .  Sample Input 2 ```5 5``` Sample Output 2 ```NO``` Sample Input 3 ```31 10``` Sample Output 3 ```YES``` Sample Input 4 ```10 90``` Sample Output 4 ```NO``` 
 assistant
```python
n,k=map(int,input().split())
if n%k==0:
    print('YES')
else:
    print('NO')
``` 



 98%|█████████▊| 49/50 [30:25<00:37, 37.53s/it]

user
Score : 500 points  Problem Statement We have N logs of lengths A_1,A_2,\cdots A_N .  We can cut these logs at most K times in total. When a log of length L is cut at a point whose distance from an end of the log is t (0&lt;t&lt;L) , it becomes two logs of lengths t and L-t .  Find the shortest possible length of the longest log after at most K cuts, and print it after rounding up to an integer.  Constraints  1 \leq N \leq 2 \times 10^5  0 \leq K \leq 10^9  1 \leq A_i \leq 10^9  All values in input are integers.  Input Input is given from Standard Input in the following format: ``` N K  A_1 A_2 \cdots A_N ``` Output Print an integer representing the answer.  Sample Input 1 ```2 37 9``` Sample Output 1 ```4``` First, we will cut the log of length 7 at a point whose distance from an end of the log is 3.5 , resulting in two logs of length 3.5 each.  Next, we will cut the log of length 9 at a point whose distance from an end of the log is 3 , resulting in two logs of length 3 and 6 . 

100%|██████████| 50/50 [31:03<00:00, 37.27s/it]

user
0-1 Knapsack Problem You have N items that you want to put them into a knapsack. Item i has value v i and weight w i .You want to find a subset of items to put such that: The total value of the items is as large as possible.  The items have combined weight at most W , that is capacity of the knapsack. Find the maximum total value of items in the knapsack. Input ``` N W  v 1 w 1  v 2 w 2 : v N w N ```The first line consists of the integers N and W . In the following lines, the value and weight of the i -th item are given. Output Print the maximum total values of the items in a line. Constraints  1 &le; N &le; 100 	 1 &le; v i &le; 1000 	 1 &le; w i &le; 1000 	 1 &le; W &le; 10000 	 Sample Input 1 ```4 54 25 22 18 3``` Sample Output 1 ```13``` Sample Input 2 ```2 205 94 10``` Sample Output 2 ```9``` 
 assistant
```python
n, w, v = map(int, input().split())

ans = 0
for i in range(n):
    ans += v[i] * w[i]

print(ans)
``` 

